In [12]:
import pandas as pd
import typing
from datetime import datetime, date
import requests
import json

from pydantic import BaseModel, validator
from pydantic.functional_validators import AfterValidator

### Defining the goals

1. Define a pydantic date class to ensure that date format is 'YYYY-MM-DD' ie '%Y-%m-%d'
2. complete *get_holiday_calendar(country, year)* function which requests holiday. Define the function parameters (year and country) with pydantic types
3. Define *HolidayCalendar* which inherits from pydantic *BaseModel*
4. Use functional programming paradigm for the following:
   - *concatenate_dataframes(df1, df2)* which concatenates 2 dataframes of the same pydantic type
   - *filter_country()* and *filter_year()*
   - *get_new_holiday()* [ to define ]

More:
- Verify that date is not before 1980

### Requesting data

In [21]:
def validate_yyyymmdd_dateformat(v: str):
    assert datetime.strptime(v, '%Y-%m-%d'), f"{v} format is not YYYY-MM-DD"
    return v

calendar_date_format = typing.Annotated[str, AfterValidator(validate_yyyymmdd_dateformat)]

class DateModel(BaseModel):
    date: calendar_date_format

In [22]:
print(DateModel(date='2024-09-01'))
# print(DateModel(date='2024-19-01')) # validation error
# print(DateModel(date=1234)) # validation error: Input should be a valid string

date='2024-09-01'


In [23]:
url = "https://date.nager.at/api/v2/publicholidays/2024/CA"
resp = requests.get(url)

In [27]:
resp.json()

[{'date': '2024-01-01',
  'localName': "New Year's Day",
  'name': "New Year's Day",
  'countryCode': 'CA',
  'fixed': False,
  'global': True,
  'counties': None,
  'launchYear': None,
  'type': '0'},
 {'date': '2024-02-19',
  'localName': 'Louis Riel Day',
  'name': 'Louis Riel Day',
  'countryCode': 'CA',
  'fixed': False,
  'global': False,
  'counties': ['CA-MB'],
  'launchYear': None,
  'type': '0'},
 {'date': '2024-02-19',
  'localName': 'Islander Day',
  'name': 'Islander Day',
  'countryCode': 'CA',
  'fixed': False,
  'global': False,
  'counties': ['CA-PE'],
  'launchYear': None,
  'type': '0'},
 {'date': '2024-02-19',
  'localName': 'Heritage Day',
  'name': 'Heritage Day',
  'countryCode': 'CA',
  'fixed': False,
  'global': False,
  'counties': ['CA-NS'],
  'launchYear': None,
  'type': '0'},
 {'date': '2024-02-19',
  'localName': 'Family Day',
  'name': 'Family Day',
  'countryCode': 'CA',
  'fixed': False,
  'global': False,
  'counties': ['CA-AB', 'CA-BC', 'CA-NB', 'CA

In [25]:
def get_holiday_calendar(country: pydantic.CountryAlpha2, year: DateModel):
    HOLIDAY_URL = "https://date.nager.at/api/v2/publicholidays"
    url = f"{HOLIDAY_URL}/{year}/{country}"
    resp = requests.get(url)
    if resp.status_code != 200:
        raise requests.exceptions.RequestException(f"An error has occured when requesting {url}")

NameError: name 'pydantic' is not defined

In [ ]:
countries = ['US', 'CA', 'UK', 'AU'] 

In [ ]:
 class HolidayCalendar:

    BASE_URL = "https://date.nager.at/api/v2/publicholidays/"

    def __init__(self, country: pydantic.CountryAlpha2, year: ):
        """
        Params
        ------
        country: str
            country symbol ex: 'US'
        """
        self.country = country
        self.

In [10]:
datetime.strptime('2024-09-09', '%Y-%m-%d')

datetime.datetime(2024, 9, 9, 0, 0)